In [21]:
import sys, os
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
sys.path.append("..")
import jkinpylib
import meshcat
import torch
import numpy as np
vis = meshcat.Visualizer()
torch.set_default_device(jkinpylib.config.DEVICE)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7002/static/


In [63]:
class Capsule():
    def __init__(self, p1, p2, r):
        super(Capsule, self).__init__()
        self.length = np.linalg.norm(p2 - p1)
        self.radius = r
        self.T = np.eye(4)
        self.T[:3, 3] = (p1 + p2) / 2
        v = p2 - p1
        v = v / np.linalg.norm(v)
        x, y, z = v
        sign = 1 if z > 0 else -1
        a = -1 / (sign + z)
        b = x * y * a
        t1 = np.array([1 + sign * x * x * a, sign * b, -sign * x])
        t2 = np.array([b, sign + y * y * a, -y])
        self.T[:3, 0] = t1
        self.T[:3, 1] = v # Meshcat uses y as axis of rotational symmetry
        self.T[:3, 2] = t2

In [78]:
vis.delete()
robot = jkinpylib.robots.Panda()

links = [
    "link0",
    "link1",
    "link2",
    "link3",
    "link4",
    "link5",
    "link6",
    "link7",
]
x = torch.zeros(1, robot.n_dofs)
for t in range(20):
    x[0, 4]= t/20*np.pi
    base_T_joints = robot.forward_kinematics_batch(
        x, return_full=True, out_device=x.device, dtype=x.dtype
    )
    T1s = base_T_joints[:, robot._collision_idx0, :, :].reshape(-1, 4, 4)
    T2s = base_T_joints[:, robot._collision_idx1, :, :].reshape(-1, 4, 4)
    c1s = (
        robot._collision_capsules[robot._collision_idx0, :]
        .expand(1, -1, -1)
        .reshape(-1, 7)
    )
    c2s = (
        robot._collision_capsules[robot._collision_idx1, :]
        .expand(1, -1, -1)
        .reshape(-1, 7)
    )

    dists = jkinpylib.geometry.capsule_capsule_distance_batch(
        c1s, T1s, c2s, T2s
    ).reshape(1, -1)

    for link_i, link in enumerate(links):
        if link is None:
            continue

        T = base_T_joints[0, link_i, :, :].cpu().numpy().astype(np.float64)
        vis[f"panda/{link}"].set_transform(T)

        vis[f"panda/{link}/mesh"].set_object(
            meshcat.geometry.DaeMeshGeometry.from_file(f"../jkinpylib/urdfs/panda/meshes/visual/{link}.dae"),
            meshcat.geometry.MeshLambertMaterial(color=0xffffff)
        )

        capsule = robot._collision_capsules[link_i, :].cpu().numpy().astype(np.float64)
        p1, p2, capsule_radius = capsule[:3], capsule[3:6], capsule[6]
        capsule_geom = Capsule(p1, p2, capsule_radius)
        color = 0x00ff00
        is_collide = torch.any(dists[0, robot._collision_idx0 == link_i] < 0) or torch.any(dists[0, robot._collision_idx1 == link_i] < 0)
        if is_collide:
            color = 0xff0000
        material = meshcat.geometry.MeshLambertMaterial(color=color, opacity=0.4)
        vis[f"panda/{link}/capsule/p1"].set_object(meshcat.geometry.Sphere(capsule_radius), material)
        vis[f"panda/{link}/capsule/p1"].set_transform(meshcat.transformations.translation_matrix(p1))
        vis[f"panda/{link}/capsule/p2"].set_object(meshcat.geometry.Sphere(capsule_radius), material)
        vis[f"panda/{link}/capsule/p2"].set_transform(meshcat.transformations.translation_matrix(p2))
        cyl_geom = meshcat.geometry.Cylinder(capsule_geom.length, capsule_radius)
        vis[f"panda/{link}/capsule/cyl"].set_object(cyl_geom, material)
        vis[f"panda/{link}/capsule/cyl"].set_transform(capsule_geom.T)


WorldModel::LoadRobot: /Users/drm/.cache/jkinpylib/temp_urdfs/panda_arm_hand_formatted_link_filepaths_absolute.urdf
joint mimic: no multiplier, using default value of 1 
joint mimic: no offset, using default value of 0 
URDFParser: Link size: 17
URDFParser: Joint size: 12
Geometry: Loading 12 meshes from /Users/drm/Projects/IKFLow/jkinpylib/scripts/../jkinpylib/urdfs/panda/meshes/visual/link0.dae into Group
Geometry: Loading 4 meshes from /Users/drm/Projects/IKFLow/jkinpylib/scripts/../jkinpylib/urdfs/panda/meshes/visual/link3.dae into Group
Geometry: Loading 4 meshes from /Users/drm/Projects/IKFLow/jkinpylib/scripts/../jkinpylib/urdfs/panda/meshes/visual/link4.dae into Group
Geometry: Loading 3 meshes from /Users/drm/Projects/IKFLow/jkinpylib/scripts/../jkinpylib/urdfs/panda/meshes/visual/link5.dae into Group
Geometry: Loading 17 meshes from /Users/drm/Projects/IKFLow/jkinpylib/scripts/../jkinpylib/urdfs/panda/meshes/visual/link6.dae into Group
Geometry: Loading 8 meshes from /Users/d

In [54]:
vis.jupyter_cell()

In [71]:
vis.open()

<Visualizer using: <meshcat.visualizer.ViewerWindow object at 0x2a4aecdc0> at path: <meshcat.path.Path object at 0x17db5a400>>